In [5]:
!pip3 install -q datasets transformers accelerate peft evaluate rouge_score 

In [10]:
!pip3 install -q -U bitsandbytes

In [9]:
!pip uninstall -y bitsandbytes

Found existing installation: bitsandbytes 0.42.0
Uninstalling bitsandbytes-0.42.0:
  Successfully uninstalled bitsandbytes-0.42.0


In [7]:
import random
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
import evaluate
from transformers import (
    GPT2TokenizerFast,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer, BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import tqdm

# ── 1) Reproducibility & Device ───────────────────────────────────────────────
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = (
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f"Using device: {device}")

# ── 2) Tokenizer ───────────────────────────────────────────────────────────────
tok = GPT2TokenizerFast.from_pretrained("gpt2")
tok.add_special_tokens({"additional_special_tokens": ["<sum>"]})
tok.pad_token = tok.eos_token
tok.padding_side = "left"

Using device: mps


In [8]:
# ── 3) Load & 4-bit Quantize Base Model ───────────────────────────────────────
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
base = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    quantization_config=bnb_config,
    device_map="auto",
)
base.resize_token_embeddings(len(tok))
base = prepare_model_for_kbit_training(base)

# ── 4) Apply LoRA adapters ────────────────────────────────────────────────────
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["c_attn"],
    task_type=TaskType.CAUSAL_LM,
)
peft_model = get_peft_model(base, lora_cfg)
peft_model.print_trainable_parameters()
peft_model.to(device)

# ── 5) Wrap in your explicit head ─────────────────────────────────────────────
class GPT2Summarizer(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        self.lm_head = base_model.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None):
        out = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True
        )
        return {"loss": out.loss, "logits": out.logits}

    def generate(self, *args, **kwargs):
        return self.base.generate(*args, **kwargs)

model = GPT2Summarizer(peft_model)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
train_raw = load_dataset("cnn_dailymail", "3.0.0", split="train[:100000]")
val_raw   = load_dataset("cnn_dailymail", "3.0.0", split="validation[:10000]")
rouge     = evaluate.load("rouge")

def preprocess(ex):
    inp = tok(
        "<sum> " + ex["article"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    lbl = tok(
        ex["highlights"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )
    return {
        "input_ids":      inp["input_ids"],
        "attention_mask": inp["attention_mask"],
        "labels":         lbl["input_ids"],
    }

train_ds = train_raw.map(preprocess, batched=False, remove_columns=train_raw.column_names)
val_ds   = val_raw.map(preprocess,   batched=False, remove_columns=val_raw.column_names)

# ── 7) Data collator ──────────────────────────────────────────────────────────
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# ── 8) Training arguments ─────────────────────────────────────────────────────
args = TrainingArguments(
    output_dir="sum-qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    fp16=(device=="cuda"),
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    logging_steps=20,
    save_strategy="no",
    report_to=[],
)

# ── 9) Trainer setup ─────────────────────────────────────────────────────────
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collator,
    tokenizer=tok,
)

# ── 10) Train the QLoRA adapter ───────────────────────────────────────────────
trainer.train()
model.base.save_pretrained("sum-qlora")  
tok.save_pretrained("sum-qlora")

/tmp/ipykernel_35/1603371352.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,4.896500,4.027040
2,4.911300,3.980206
3,4.914000,3.944669


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('sum-qlora/tokenizer_config.json',
 'sum-qlora/special_tokens_map.json',
 'sum-qlora/vocab.json',
 'sum-qlora/merges.txt',
 'sum-qlora/added_tokens.json',
 'sum-qlora/tokenizer.json')

In [ ]:
preds, refs = [], []
for ex in val_raw.select(range(10000)):
    inputs = tok(
        "<sum> " + ex["article"],
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding=True,
    ).input_ids.to(device)

    output = peft_model.generate(
        inputs,
        max_new_tokens=80,
        num_beams=4,
        length_penalty=1.2,
        early_stopping=True,
        pad_token_id=tok.eos_token_id,
    )[0]

    summary = tok.decode(output[inputs.shape[1]:], skip_special_tokens=True).strip()
    preds.append(summary)
    refs.append(ex["highlights"])

result = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
print("Final ROUGE:", result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Final ROUGE: {'rouge1': 0.06362446663445848, 'rouge2': 0.00625, 'rougeL': 0.05802836052801211, 'rougeLsum': 0.05643305706937529}
